In [ ]:
from fastai.vision.all import *

In [ ]:
! git clone --depth 1 https://github.com/rwightman/pytorch-image-models.git

Cloning into 'pytorch-image-models'...
remote: Enumerating objects: 576, done.
remote: Counting objects: 100% (576/576), done.
remote: Compressing objects: 100% (404/404), done.
remote: Total 576 (delta 225), reused 390 (delta 166), pack-reused 0
Receiving objects: 100% (576/576), 2.55 MiB | 14.98 MiB/s, done.
Resolving deltas: 100% (225/225), done.


In [ ]:
%cd pytorch-image-models

/content/pytorch-image-models


In [ ]:
import pandas as pd


[Errno 2] No such file or directory: 'pytorch-image-models/results'
/content/pytorch-image-models


In [ ]:
!pwd

/content/pytorch-image-models


In [ ]:
%cd results


/content/pytorch-image-models/results


In [ ]:
!ls

benchmark-infer-amp-nchw-pt111-cu113-rtx3090.csv
benchmark-infer-amp-nchw-pt112-cu113-rtx3090.csv
benchmark-infer-amp-nchw-pt113-cu117-rtx3090.csv
benchmark-infer-amp-nchw-pt210-cu121-rtx3090.csv
benchmark-infer-amp-nhwc-pt111-cu113-rtx3090.csv
benchmark-infer-amp-nhwc-pt112-cu113-rtx3090.csv
benchmark-infer-amp-nhwc-pt113-cu117-rtx3090.csv
benchmark-infer-amp-nhwc-pt210-cu121-rtx3090.csv
benchmark-infer-fp32-nchw-pt221-cpu-i9_10940x-dynamo.csv
benchmark-train-amp-nchw-pt111-cu113-rtx3090.csv
benchmark-train-amp-nchw-pt112-cu113-rtx3090.csv
benchmark-train-amp-nhwc-pt111-cu113-rtx3090.csv
benchmark-train-amp-nhwc-pt112-cu113-rtx3090.csv
generate_csv_results.py
model_metadata-in1k.csv
README.md
results-imagenet-a-clean.csv
results-imagenet-a.csv
results-imagenet.csv
results-imagenet-r-clean.csv
results-imagenet-r.csv
results-imagenet-real.csv
results-imagenetv2-matched-frequency.csv
results-sketch.csv


In [ ]:
df_results = pd.read_csv('results-imagenet.csv')

In [ ]:
df_results['model_org'] = df_results['model']
df_results['model'] = df_results['model'].str.split('.').str[0]

In [ ]:
df_results.head()

,model,top1,top1_err,top5,top5_err,param_count,img_size,crop_pct,interpolation,model_org
0,eva02_large_patch14_448,90.052,9.948,99.048,0.952,305.08,448,1.0,bicubic,eva02_large_patch14_448.mim_m38m_ft_in22k_in1k
1,eva02_large_patch14_448,89.970,10.030,99.012,0.988,305.08,448,1.0,bicubic,eva02_large_patch14_448.mim_in22k_ft_in22k_in1k
2,eva_giant_patch14_560,89.786,10.214,98.992,1.008,"1,014.45",560,1.0,bicubic,eva_giant_patch14_560.m30m_ft_in22k_in1k
3,eva02_large_patch14_448,89.622,10.378,98.950,1.050,305.08,448,1.0,bicubic,eva02_large_patch14_448.mim_in22k_ft_in1k
4,eva02_large_patch14_448,89.574,10.426,98.924,1.076,305.08,448,1.0,bicubic,eva02_large_patch14_448.mim_m38m_ft_in1k


In [ ]:
def get_data(part, col):
    df = pd.read_csv(f'benchmark-{part}-amp-nhwc-pt111-cu113-rtx3090.csv').merge(df_results, on='model')
    df['secs'] = 1. / df[col]
    df['family'] = df.model.str.extract('^([a-z]+?(?:v2)?)(?:\d|_|$)')
    df = df[~df.model.str.endswith('gn')]
    df.loc[df.model.str.contains('in22'),'family'] = df.loc[df.model.str.contains('in22'),'family'] + '_in22'
    df.loc[df.model.str.contains('resnet.*d'),'family'] = df.loc[df.model.str.contains('resnet.*d'),'family'] + 'd'
    return df[df.family.str.contains('^re[sg]netd?|beit|convnext|levit|efficient|vit|vgg|swin')]

In [ ]:


df = get_data('infer', 'infer_samples_per_sec')




In [ ]:
import plotly.express as px
w, h = 1000, 800


In [ ]:
def show_all(df, title, size):
  return px.scatter(df, width=w, height=h, size=df[size]**2, title=title, x='secs',
  y='top1', log_x=True, color='family', hover_name='model_org', hover_data=[size])

In [ ]:
show_all(df, 'Inference', 'infer_img_size')

ModuleNotFoundError: No module named 'pytorch_image_models'